In [1]:
# General imports
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle


# Modelling Imports
import sklearn
import sklearn.tree
import sklearn.metrics
import sklearn.naive_bayes
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import sklearn.ensemble
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost

from simpletransformers.classification import ClassificationModel

# Useful relative paths
data_directory = './data/NELA'
model_directory ='./models'

//anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
I1201 19:10:43.732488 4555400640 __init__.py:43] Loading faiss.
In Transformers v4.0.0, the default path to cache downloaded models changed from '~/.cache/torch/transformers' to '~/.cache/huggingface/transformers'. Since you don't seem to have overridden and '~/.cache/torch/transformers' is a directory that exists, we're moving it to '~/.cache/huggingface/transformers' to avoid redownloading models you have already in the cache. You should only see this message once.


# 1. – Modelling! Most, if not all, of this section draws heavily from the [HuggingFace documentation and source code](https://huggingface.co/transformers/model_doc/electra.html)

## 1.1 Lets make data a bit more BERT/Electra friendly

In [5]:
X = pd.read_csv('{}/complete_processed.csv'.format(data_directory))

In [8]:
X=X[['full_preprocessed','label']]
y=X['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [14]:
X_train.to_csv(('{}/BERT_data/train.csv'.format(data_directory)),index=False)
X_test.to_csv(('{}/BERT_data/test.csv'.format(data_directory)),index=False)

In [18]:
model_type='electra'
model_name='google/electra-small-discriminator'

In [24]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 3,
    "evaluate_during_training_steps": 1000,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}


In [25]:
model = ClassificationModel(model_type, model_name, args=train_args, use_cuda=False)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['pooler.den

In [26]:
train_df = pd.read_csv('{}/BERT_data/train.csv'.format(data_directory))
eval_df = pd.read_csv('{}/BERT_data/test.csv'.format(data_directory))

In [27]:
model.train_model(train_df, eval_df=eval_df)

//anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:380: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
I1201 19:37:41.257708 4555400640 classification_model.py:1082]  Converting to features started. Cache is not used.


KeyboardInterrupt: 